HDFS - kopiowanie danych

In [1]:
import os 
user_name = os.environ.get('USER')
print (user_name)

lksiezak


In [15]:
%%bash
cd ~
mkdir -p data
cd data
cp /data/local/datascience/data/brca.txt .
head brca.txt
wc -l brca.txt

Sample	Chromosome	Start	End	Num_Probes	Segment_Mean
TCGA-A2-A0EU-10A-01D-A060-02	1	10209	2583075	NA	-0.210121164888764
TCGA-A2-A0EU-10A-01D-A060-02	1	2583076	249240606	NA	-0.101789555120838
TCGA-A2-A0EU-10A-01D-A060-02	2	10002	243189359	NA	-0.337963741969504
TCGA-A2-A0EU-10A-01D-A060-02	3	60175	162511435	NA	-0.0850647472883194
TCGA-A2-A0EU-10A-01D-A060-02	3	162511436	162626067	NA	0.42309157125509
TCGA-A2-A0EU-10A-01D-A060-02	3	162626068	197962416	NA	-0.0962225847981023
TCGA-A2-A0EU-10A-01D-A060-02	4	69223	191014397	NA	-0.0713350286479217
TCGA-A2-A0EU-10A-01D-A060-02	5	11770	180905246	NA	0.176064461471459
TCGA-A2-A0EU-10A-01D-A060-02	6	63815	171050932	NA	-0.0895478602860139
5299 brca.txt


ls => hdfs dfs -ls
cp => hdfs dfs -cp
rm => hdfs dfs -rm

In [24]:
%%bash
hdfs dfs -ls

hdfs dfs -mkdir -p hdfs:///user/${USER}/data
hdfs dfs -put -f brca.txt hdfs:///user/${USER}/data

Found 2 items
-rw-r--r--   2 lksiezak bdg-ds-stud     375612 2020-01-11 09:49 brca.txt
drwxr-xr-x   - lksiezak bdg-ds-stud          0 2020-01-11 09:51 data


In [23]:
%%bash
hdfs dfs -ls hdfs:///user/${USER}/data

Found 1 items
-rw-r--r--   2 lksiezak bdg-ds-stud     375612 2020-01-11 09:51 hdfs:///user/lksiezak/data/brca.txt


In [25]:
!hdfs dfs -cat /user/${USER}/data/brca.txt | head

Sample	Chromosome	Start	End	Num_Probes	Segment_Mean
TCGA-A2-A0EU-10A-01D-A060-02	1	10209	2583075	NA	-0.210121164888764
TCGA-A2-A0EU-10A-01D-A060-02	1	2583076	249240606	NA	-0.101789555120838
TCGA-A2-A0EU-10A-01D-A060-02	2	10002	243189359	NA	-0.337963741969504
TCGA-A2-A0EU-10A-01D-A060-02	3	60175	162511435	NA	-0.0850647472883194
TCGA-A2-A0EU-10A-01D-A060-02	3	162511436	162626067	NA	0.42309157125509
TCGA-A2-A0EU-10A-01D-A060-02	3	162626068	197962416	NA	-0.0962225847981023
TCGA-A2-A0EU-10A-01D-A060-02	4	69223	191014397	NA	-0.0713350286479217
TCGA-A2-A0EU-10A-01D-A060-02	5	11770	180905246	NA	0.176064461471459
TCGA-A2-A0EU-10A-01D-A060-02	6	63815	171050932	NA	-0.0895478602860139
cat: Unable to write to output stream.


In [26]:
!hdfs dfs -du -h /user/${USER}/data/brca.txt

366.8 K  733.6 K  /user/lksiezak/data/brca.txt


^ Powyzszy wynik wskazuje że plik jest trzymany w dwóch kopiach

In [27]:
!hdfs dfs -mkdir -p /user/${USER}/data/external 

In [29]:
!hdfs dfs -cp -f /user/${USER}/data/brca.txt /user/${USER}/data/external/brca.txt 

cp: `/user/lksiezak/data/external/brca.txt': File exists


In [31]:
!hdfs dfs -ls -R /user/${USER}/data

-rw-r--r--   2 lksiezak bdg-ds-stud     375612 2020-01-11 09:56 /user/lksiezak/data/brca.txt
drwxr-xr-x   - lksiezak bdg-ds-stud          0 2020-01-11 10:09 /user/lksiezak/data/external
-rw-r--r--   2 lksiezak bdg-ds-stud     375612 2020-01-11 10:09 /user/lksiezak/data/external/brca.txt


Usunięcie powoduje tak naprawdę przeniesienie pliku do folderu Trash: hdfs://cdh01.cl.ii.pw.edu.pl:8020/user/lksiezak/.Trash/Current/user/lksiezak/data/brca.txt 
Tam będzie czekał na automatyczne usunięcie przez 30 dni 

In [32]:
! hdfs dfs -rm /user/${USER}/data/brca.txt

20/01/11 10:12:00 INFO fs.TrashPolicyDefault: Moved: 'hdfs://cdh01.cl.ii.pw.edu.pl:8020/user/lksiezak/data/brca.txt' to trash at: hdfs://cdh01.cl.ii.pw.edu.pl:8020/user/lksiezak/.Trash/Current/user/lksiezak/data/brca.txt


In [34]:
!hdfs dfs -ls hdfs:///user/${USER}/data

Found 1 items
drwxr-xr-x   - lksiezak bdg-ds-stud          0 2020-01-11 10:09 hdfs:///user/lksiezak/data/external


In [ ]:
Pobranie pliku z hdfsa do systemu lokalnego

In [35]:
!hdfs dfs -get /user/${USER}/data/external/brca.txt ~

In [37]:
!hdfs fsck /user/${USER}/data/external/brca.txt

Connecting to namenode via http://cdh01.cl.ii.pw.edu.pl:50070/fsck?ugi=lksiezak&path=%2Fuser%2Flksiezak%2Fdata%2Fexternal%2Fbrca.txt
FSCK started by lksiezak (auth:KERBEROS_SSL) from /192.168.157.200 for path /user/lksiezak/data/external/brca.txt at Sat Jan 11 10:17:17 CET 2020

Status: HEALTHY
 Number of data-nodes:	4
 Number of racks:		1
 Total dirs:			0
 Total symlinks:		0

Replicated Blocks:
 Total size:	375612 B
 Total files:	1
 Total blocks (validated):	1 (avg. block size 375612 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	0 (0.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	2
 Average block replication:	2.0
 Missing blocks:		0
 Corrupt blocks:		0
 Missing replicas:		0 (0.0 %)

Erasure Coded Block Groups:
 Total size:	0 B
 Total files:	0
 Total block groups (validated):	0
 Minimally erasure-coded block groups:	0
 Over-erasure-coded block groups:	0
 Under-erasure-coded block groups:	0
 Unsatisfactory

# Spark SQL

In [38]:
import random
port_name = random.randint(4000, 4999)
port_name

4457

Tworzymy sesję sparka:

In [40]:
from pyspark.sql import SparkSession

spark = SparkSession \
.builder \
.master('yarn-client') \
.config('spark.driver.memory', '1g') \
.config('spark.executor.memory', '1g') \
.config('spark.ui.port', port_name) \
.appName(f'ds_{user_name}') \
.getOrCreate()


In [42]:
path = f'/user/{user_name}/data/external/brca.txt'
path

'/user/lksiezak/data/external/brca.txt'

In [43]:
spark.sql('show databases').show()

+----------------+
|    databaseName|
+----------------+
|        alewalsk|
|          am0022|
|        arutkows|
|        arydzew1|
|          as0030|
|           awisz|
|   bdgz_platform|
|          bh0012|
|        bkruszy1|
|     datascience|
|       db_po0023|
|          dborys|
|         default|
|          dk0019|
|         dkl0017|
|          dl0021|
|ds_lab_testuser1|
|ds_lab_testuser2|
|          ec0004|
|  edugen_sequila|
+----------------+
only showing top 20 rows



In [48]:
spark.sql(f'DROP DATABASE {user_name} CASCADE')
spark.sql(f"CREATE DATABASE {user_name} LOCATION '/edugen/db/{user_name}'")

DataFrame[]

In [49]:
spark.sql(f'USE {user_name}')

DataFrame[]

In [50]:
table_name = 'brac'
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_name} \
            USING  csv \
            OPTIONS (DELIMITER "\t", HEADER true, INFERSCHEMA true) \
            LOCATION  "{path}"')

DataFrame[]

In [51]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|lksiezak|     brac|      false|
+--------+---------+-----------+



In [54]:
spark.sql(f"DESCRIBE {table_name}").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|              Sample|              string|   null|
|          Chromosome|                 int|   null|
|               Start|                 int|   null|
|                 End|                 int|   null|
|          Num_Probes|              string|   null|
|        Segment_Mean|              double|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|            lksiezak|       |
|               Table|                brac|       |
|               Owner|lksiezak@CL.II.PW...|       |
|        Created Time|Sat Jan 11 11:13:...|       |
|         Last Access|Thu Jan 01 01:00:...|       |
|          Created By|         Spark 2.4.3|       |
|                Type|            EXTERNAL|       |
|            Provider|                 csv|       |
|    Table P

In [ ]:
spark.sql(f"DESCRIBE extended {table_name}").show()

In [55]:
spark.sql(f"SELECT count(*) from {table_name}").show()

+--------+
|count(1)|
+--------+
|    5298|
+--------+



In [58]:
spark.sql(f"SELECT * FROM {table_name}").show(truncate=False)

+----------------------------+----------+---------+---------+----------+-------------------+
|Sample                      |Chromosome|Start    |End      |Num_Probes|Segment_Mean       |
+----------------------------+----------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-01D-A060-02|1         |10209    |2583075  |NA        |-0.210121164888764 |
|TCGA-A2-A0EU-10A-01D-A060-02|1         |2583076  |249240606|NA        |-0.101789555120838 |
|TCGA-A2-A0EU-10A-01D-A060-02|2         |10002    |243189359|NA        |-0.337963741969504 |
|TCGA-A2-A0EU-10A-01D-A060-02|3         |60175    |162511435|NA        |-0.0850647472883194|
|TCGA-A2-A0EU-10A-01D-A060-02|3         |162511436|162626067|NA        |0.42309157125509   |
|TCGA-A2-A0EU-10A-01D-A060-02|3         |162626068|197962416|NA        |-0.0962225847981023|
|TCGA-A2-A0EU-10A-01D-A060-02|4         |69223    |191014397|NA        |-0.0713350286479217|
|TCGA-A2-A0EU-10A-01D-A060-02|5         |11770    |180905246|NA       

- Wyswietl unikalne nazwy chromosomow 
- Ile ich jest

In [64]:
spark.sql(f"SELECT distinct Chromosome FROM {table_name}").show(truncate=False)

+----------+
|Chromosome|
+----------+
|12        |
|22        |
|1         |
|13        |
|6         |
|16        |
|3         |
|20        |
|5         |
|19        |
|15        |
|9         |
|17        |
|4         |
|8         |
|23        |
|7         |
|10        |
|24        |
|21        |
+----------+
only showing top 20 rows



In [67]:
spark.sql(f"SELECT COUNT(DISTINCT Chromosome) FROM {table_name}").show(truncate=False)

+--------------------------+
|count(DISTINCT Chromosome)|
+--------------------------+
|24                        |
+--------------------------+



In [69]:
spark.sql(f"SELECT DISTINCT Sample FROM {table_name}").show(truncate=False)

+----------------------------+
|Sample                      |
+----------------------------+
|TCGA-BH-A0H7-10A-01D-A060-02|
|TCGA-A2-A0EU-10A-01D-A060-02|
|TCGA-AO-A0JL-01A-11D-A060-02|
|TCGA-AO-A0JF-10A-01D-A060-02|
|TCGA-AR-A0TU-01A-31D-A106-02|
|TCGA-BH-A0BM-01A-11D-A060-02|
|TCGA-A7-A0D9-01A-31D-A060-02|
|TCGA-A7-A0D9-10A-01D-A060-02|
|TCGA-BH-A0W5-10A-01D-A106-02|
|TCGA-AO-A0JJ-01A-11D-A060-02|
|TCGA-BH-A0DK-10A-01D-A060-02|
|TCGA-BH-A0GY-01A-11D-A060-02|
|TCGA-B6-A0RI-10A-01D-A060-02|
|TCGA-BH-A0HB-10A-01D-A060-02|
|TCGA-B6-A0RI-01A-11D-A060-02|
|TCGA-AO-A0JF-01A-11D-A060-02|
|TCGA-BH-A0H6-01A-21D-A060-02|
|TCGA-B6-A0X4-01A-11D-A106-02|
|TCGA-AR-A0TU-10A-01D-A106-02|
|TCGA-BH-A0HX-10A-02D-A060-02|
+----------------------------+
only showing top 20 rows



In [70]:
spark.sql(f"SELECT * FROM {table_name} WHERE sample='TCGA-BH-A0W5-10A-01D-A106-02'").show()

+--------------------+----------+--------+---------+----------+-------------------+
|              Sample|Chromosome|   Start|      End|Num_Probes|       Segment_Mean|
+--------------------+----------+--------+---------+----------+-------------------+
|TCGA-BH-A0W5-10A-...|         1|   10208|249240606|        NA|-0.0342829456904825|
|TCGA-BH-A0W5-10A-...|         2|   10001|243189359|        NA|-0.0361339272866173|
|TCGA-BH-A0W5-10A-...|         3|   60174|197962416|        NA|-0.0353001445469591|
|TCGA-BH-A0W5-10A-...|         4|   68818|191014415|        NA|-0.0381610776778173|
|TCGA-BH-A0W5-10A-...|         5|   11769|180905246|        NA| 0.0945120717312347|
|TCGA-BH-A0W5-10A-...|         6|   63814|171050932|        NA|-0.0370055036510112|
|TCGA-BH-A0W5-10A-...|         7|   10127|159128640|        NA| -0.036651266453263|
|TCGA-BH-A0W5-10A-...|         8|   23379| 37177805|        NA| -0.296138511854961|
|TCGA-BH-A0W5-10A-...|         8|37177806|146303846|        NA|  0.394414200

Podaj liczbę wierszy dla każdej grupy oraz maksymlną wartość średniej segmentu dla chromosomow od 1 do 5

In [80]:
spark.sql(f"SELECT sample, COUNT(*), MAX(Segment_Mean) FROM {table_name} WHERE Chromosome <= 5 GROUP BY Sample").show(truncate=False)

+----------------------------+--------+------------------+
|sample                      |count(1)|max(Segment_Mean) |
+----------------------------+--------+------------------+
|TCGA-BH-A0H7-10A-01D-A060-02|116     |0.882987851355116 |
|TCGA-A2-A0EU-10A-01D-A060-02|8       |0.42309157125509  |
|TCGA-AO-A0JL-01A-11D-A060-02|127     |3.82790441644727  |
|TCGA-AO-A0JF-10A-01D-A060-02|6       |0.289237261179585 |
|TCGA-AR-A0TU-01A-31D-A106-02|71      |1.32969350720596  |
|TCGA-BH-A0BM-01A-11D-A060-02|10      |0.398470909941123 |
|TCGA-A7-A0D9-01A-31D-A060-02|5       |-0.196448961894486|
|TCGA-A7-A0D9-10A-01D-A060-02|5       |-0.196448961894486|
|TCGA-BH-A0W5-10A-01D-A106-02|5       |0.0945120717312347|
|TCGA-AO-A0JJ-01A-11D-A060-02|6       |0.358494442304096 |
|TCGA-BH-A0DK-10A-01D-A060-02|15      |0.394393774576986 |
|TCGA-BH-A0GY-01A-11D-A060-02|31      |1.24033661769591  |
|TCGA-B6-A0RI-10A-01D-A060-02|9       |0.481228280412584 |
|TCGA-BH-A0HB-10A-01D-A060-02|66      |0.974774612914482

Zwroc chromosomy dla ktorych srednia wartosc segmentu jest wieksza niz 0.05 w zakresie pozycji startowych 100-10000

In [93]:
spark.sql(f"SELECT Chromosome, MEAN(segment_mean) FROM {table_name} WHERE Start between 100 and 100000 GROUP BY Chromosome HAVING MEAN(segment_mean)>0.05").show(truncate=False)

+----------+-------------------+
|Chromosome|avg(segment_mean)  |
+----------+-------------------+
|12        |0.995365981884277  |
|16        |0.12415591706179774|
|20        |0.11056720549428493|
|10        |0.12244174812163248|
+----------+-------------------+

